In [1]:
import os
import folium
import matplotlib as plt
import matplotlib.colors as mcolors
import seaborn as sns
import polars as pl
from data_io.loader.data_loader import DataLoader

In [2]:
dl = DataLoader()

# Heidelberg
acc = (
    dl.accident_data
    .bicycle_only()
    .filter_region(state=8, region=2, district=21)
)

points = acc.map_points()

In [3]:
years = sorted(acc.df["year"].unique().to_list())
cmap = plt.colormaps['viridis']
year_colors = {
    year: mcolors.rgb2hex(cmap(i / (len(years)-1)))
    for i, year in enumerate(years)
}

In [4]:
center = acc.df.select(["latitude", "longitude"]).mean().to_numpy()[0].tolist()

m = folium.Map(location=center, zoom_start=13)

for lat, lon, year, month in points:
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        popup=f"Year: {year}, Month: {month}",
        color=year_colors[year],
        fill=True,
        fill_color=year_colors[year],
        fill_opacity=0.7
    ).add_to(m)

legend_html = '''
     <div style="
         position: fixed;
         bottom: 50px;
         left: 50px;
         width: 150px;
         height: auto;
         border: 2px solid grey;
         z-index: 9999;
         background-color: white;
         padding: 10px;
         font-size: 14px;
     ">
     <b>Accident Years</b><br>
'''

for year in years:
    color = year_colors[year]
    legend_html += f'''
        <i style="
            background:{color};
            width:12px;
            height:12px;
            float:left;
            margin-right:6px;
            opacity:0.9;
        "></i>{year}<br>
    '''

legend_html += "</div>"

m.get_root().html.add_child(folium.Element(legend_html))

m